Nesse notebook vamos tratar os dados dos jogadores e criar features novas com bases nos dados que temos

In [17]:
from database.bq_connection import execute
from datetime import date, datetime
import pandas as pd
import os

pd.set_option('display.max_columns', 350)

In [18]:
def get_project_root_path():
    project_directory = "fifa22-players-analysis"
    separated_path = os.getcwd().split(os.sep)
    project_directory_index = separated_path.index(project_directory) + 1
    return os.sep.join(separated_path[0:project_directory_index])

In [19]:
query = """SELECT * FROM `puc-sp.fifa.player`"""

file_key_path = os.sep.join([get_project_root_path(), "database", "filekey.key"])
raw_data = execute(query, file_key_path)

In [20]:
data = raw_data
print(data.shape)
data

(19239, 91)


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,GK,76,76,NaN,NaN,1987-01-27,88,191,NaN,None,NaN,Left,3,1,1,Medium/Medium,Lean (185+),NaN,None,Team Player,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,28+2,28+2,28+2,27,28,28,28,27,29+2,29+2,29+2,28+2,29+2,29+2,29+2,28+2,27+2,29+2,29+2,29+2,27+2,27+2,28+2,28+2,28+2,27+2,75+1,Russia
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,GK,76,76,NaN,NaN,1987-10-23,84,196,NaN,None,NaN,Right,2,1,1,Medium/Medium,Normal (185+),NaN,None,"Leadership, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,33+2,33+2,33+2,31,33,33,33,31,35+2,35+2,35+2,34+2,37+2,37+2,37+2,34+2,31+2,37+2,37+2,37+2,31+2,31+2,35+2,35+2,35+2,31+2,75+1,Canada
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,GK,74,74,NaN,NaN,1988-03-17,84,187,NaN,None,NaN,Right,3,1,1,Medium/Medium,Normal (185+),NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,22+2,22+2,22+2,17,19,19,19,17,20+2,20+2,20+2,19+2,21+2,21+2,21+2,19+2,19+2,22+2,22+2,22+2,19+2,18+2,22+2,22+2,22+2,18+2,73+1,Russia
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,GK,72,72,NaN,NaN,1993-03-21,89,197,NaN,None,NaN,Right,3,1,1,Medium/Medium,Normal (185+),NaN,None,"GK Long Throw, Cautious With Crosses",NaN,NaN,NaN,NaN,NaN,NaN,15,13,18,16,14,12,25,16,48,45,50,70,36,51,37,33,64,13,39,19,14,45,14,61,19,14,15,70,70,68,71,72,47.0,28+2,28+2,28+2,26,28,28,28,26,27+2,27+2,27+2,27+2,27+2,27+2,27+2,27+2,26+2,27+2,27+2,27+2,26+2,26+2,27+2,27+2,27+2,26+2,71+1,Finland
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,GK,72,82,NaN,NaN,1999-02-25,82,192,NaN,None,NaN,Right,2,1,1,Medium/Medium,Normal (185+),NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,13,7,14,10,11,12,26,18,19,22,33,38,26,55,69,26,64,8,27,13,7,55,11,55,25,13,14,74,74,73,75,76,20.0,21+2,21+2,21+2,19,21,21,21,19,23+2,23+2,23+2,21+2,25+2,25+2,25+2,21+2,19+2,24+2,24+2,24+2,19+2,19+2,25+2,25+2,25+2,19+2,71+2,Russia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,262593,https://sofifa.com/player/262593/moritz-seiffe...,M. Seiffert,Moritz Seiffert,LW,58,66,450000.0,600.0,2000-11-04,74,180,115820.0,SUB,23.0,Left,3,2,1,Medium/Medium,Lean (170-185),743000.0,None,None,75.0,54.0,54.0,62.0,28.0,49.0,58,59,53,63,52,4

Nesse dataset temos apenas `dob` (data de nascimento) dos jogadores, vamos adicionar uma coluna com as idades dos jogadores

In [21]:
current_date = datetime.today()
data.dob = pd.to_datetime(data.dob)

def age(dob):
    today = date.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

data["age"] = data.dob.apply(age)
data[["dob", "age"]].head()

,dob,age
0,1987-01-27,35
1,1987-10-23,34
2,1988-03-17,34
3,1993-03-21,29
4,1999-02-25,23


Tambem vamos adicionar uma coluna com o Indice de massa corporal dos jogadores

In [22]:
jogadores: list = []


def valida_peso(p):
    return 0 < p < 200


def valida_altura(a):
    return 0 < a < 3


def calcula_imc(peso: float, altura: float) -> float:
    return peso / (altura ** 2)


def verifica_obesidade(imc: float) -> bool:
    return imc >= 30


def verifica_magreza(imc: float) -> bool:
    return imc <= 20


def verifica_normal(imc: float) -> bool:
    return 20 < imc < 30


def classifica_jogador(peso: float, altura: float) -> None:
    global jogadores
    if valida_altura(altura) and valida_peso(peso):
        imc = calcula_imc(peso, altura)
        obesidade = verifica_obesidade(imc)
        magreza = verifica_magreza(imc)
        normal = verifica_normal(imc)
        valido = True
    else:
        raise ValueError("Peso ou altura inválidos")
    jogadores.append((imc, obesidade, magreza, normal, valido))



for row in data.itertuples():
    classifica_jogador(row.weight_kg, row.height_cm / 100)

jogadores_df = pd.DataFrame(jogadores, columns=["imc", "obesidade", "magreza", "normal", "valido"]).head()

In [23]:
data["BMI"] = jogadores_df.imc
data["obese"] = jogadores_df.obesidade.astype(int)
data["lean"] = jogadores_df.magreza.astype(int)
data["normal"] = jogadores_df.normal.astype(int)

In [24]:
data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,BMI,obese,lean,normal
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,GK,76,76,NaN,NaN,1987-01-27,88,191,NaN,None,NaN,Left,3,1,1,Medium/Medium,Lean (185+),NaN,None,Team Player,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,28+2,28+2,28+2,27,28,28,28,27,29+2,29+2,29+2,28+2,29+2,29+2,29+2,28+2,27+2,29+2,29+2,29+2,27+2,27+2,28+2,28+2,28+2,27+2,75+1,Russia,35,24.122146,0.0,0.0,1.0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,GK,76,76,NaN,NaN,1987-10-23,84,196,NaN,None,NaN,Right,2,1,1,Medium/Medium,Normal (185+),NaN,None,"Leadership, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,33+2,33+2,33+2,31,33,33,33,31,35+2,35+2,35+2,34+2,37+2,37+2,37+2,34+2,31+2,37+2,37+2,37+2,31+2,31+2,35+2,35+2,35+2,31+2,75+1,Canada,34,21.865889,0.0,0.0,1.0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,GK,74,74,NaN,NaN,1988-03-17,84,187,NaN,None,NaN,Right,3,1,1,Medium/Medium,Normal (185+),NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,22+2,22+2,22+2,17,19,19,19,17,20+2,20+2,20+2,19+2,21+2,21+2,21+2,19+2,19+2,22+2,22+2,22+2,19+2,18+2,22+2,22+2,22+2,18+2,73+1,Russia,34,24.021276,0.0,0.0,1.0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,GK,72,72,NaN,NaN,1993-03-21,89,197,NaN,None,NaN,Right,3,1,1,Medium/Medium,Normal (185+),NaN,None,"GK Long Throw, Cautious With Crosses",NaN,NaN,NaN,NaN,NaN,NaN,15,13,18,16,14,12,25,16,48,45,50,70,36,51,37,33,64,13,39,19,14,45,14,61,19,14,15,70,70,68,71,72,47.0,28+2,28+2,28+2,26,28,28,28,26,27+2,27+2,27+2,27+2,27+2,27+2,27+2,27+2,26+2,27+2,27+2,27+2,26+2,26+2,27+2,27+2,27+2,26+2,71+1,Finland,29,22.932825,0.0,0.0,1.0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,GK,72,82,NaN,NaN,1999-02-25,82,192,NaN,None,NaN,Right,2,1,1,Medium/Medium,Normal (185+),NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,13,7,14,10,11,12,26,18,19,22,33,38,26,55,69,26,64,8,27,13,7,55,11,55,25,13,14,74,74,73,75,76,20.0,21+2,21+2,21+2,19,21,21,21,19,23+2,23+2,23+2,21+2,25+2,25+2,25+2,21+2,19+2,24+2,24+2,24+2,19+2,19+2,25+2,25+2,25+2,19+2,71+2,Russia,23,22.243924,0.0,0.0,1.0


Outra tratativa a ser feita é transformar cada `player_traits` e cada `player_tags` diferente em uma coluna boleana

In [25]:
data[["player_traits", "player_tags", "player_positions"]]

,player_traits,player_tags,player_positions
0,Team Player,None,GK
1,"Leadership, Comes For Crosses",None,GK
2,None,None,GK
3,"GK Long Throw, Cautious With Crosses",None,GK
4,None,None,GK
...,...,...,...
19234,None,None,LW
19235,None,None,RW
19236,None,None,ST
19237,None,None,ST


In [26]:
data.loc[data["player_tags"].astype(str).str.contains(r" ", regex=True)]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,BMI,obese,lean,normal
2295,215914,https://sofifa.com/player/215914/ngolo-kante/2...,N. Kanté,N'Golo Kanté,"CDM, CM",90,90,100000000.0,230000.0,1991-03-29,70,168,5.0,RCM,7.0,Right,3,2,4,Medium/High,Unique,185000000.0,"#Tackling, #Tactician",Leadership,78.0,66.0,75.0,82.0,87.0,83.0,68,65,54,79,49,49,79,81,82,75,82,93,92,71,77,97,72,65,93,91,72,78,54,84,90,93,86,15,12,10,7,10,NaN,72+3,72+3,72+3,77,77,77,77,77,78+3,78+3,78+3,79+3,82+3,82+3,82+3,79+3,85+3,87+3,87+3,87+3,85+3,85+3,84+3,84+3,84+3,85+3,20+3,France,31,NaN,NaN,NaN,NaN
2297,164240,https://sofifa.com/player/164240/thiago-emilia...,Thiago Silva,Thiago Emiliano da Silva,CB,85,85,9500000.0,105000.0,1984-09-22,79,183,5.0,SUB,6.0,Right,3,2,4,Medium/High,Normal (170-185),17600000.0,#Tactician,"Leadership, Power Header",53.0,54.0,72.0,72.0,86.0,78.0,60,40,81,67,62,61,80,79,52,54,67,82,66,71,88,68,82,65,76,88,59,71,60,86,87,86,84,9,12,5,9,10,NaN,65+3,65+3,65+3,65,67,67,67,65,69+3,69+3,69+3,67+3,75+3,75+3,75+3,67+3,76+3,82+3,82+3,82+3,76+3,77+3,83+2,83+2,83+2,77+3,18+3,Brazil,37,NaN,NaN,NaN,NaN
2298,205498,https://sofifa.com/player/205498/luiz-frello-f...,Jorginho,Luiz Frello Filho Jorge,"CM, CDM",85,85,52500000.0,170000.0,1991-12-20,68,180,5.0,LCM,5.0,Right,3,3,2,Medium/Medium,Lean (170-185),97100000.0,#Playmaker,Playmaker (AI),55.0,67.0,86.0,81.0,73.0,73.0,75,64,56,79,75,76,89,87,61,50,77,82,72,71,58,87,66,62,79,81,69,88,91,81,67,81,71,14,7,13,14,8,NaN,70+2,70+2,70+2,76,76,76,76,76,80+2,80+2,80+2,78+2,84+1,84+1,84+1,78+2,78+2,82+2,82+2,82+2,78+2,76+2,73+2,73+2,73+2,76+2,19+2,Italy,30,NaN,NaN,NaN,NaN
2388,209499,https://sofifa.com/player/209499/fabio-henriqu...,Fabinho,Fábio Henrique Tavares,"CDM, CB",86,88,73500000.0,165000.0,1993-10-23,78,188,9.0,CDM,3.0,Right,2,3,3,Medium/High,Unique,141500000.0,"#Tackling, #Tactician","Dives Into Tackles (AI), Long Passer (AI)",67.0,69.0,78.0,78.0,85.0,83.0,75,63,75,78,71,53,82,81,65,68,64,86,64,81,75,88,81,70,85,87,70,75,90,84,86,87,85,13,12,6,8,8,NaN,74+3,74+3,74+3,75,75,75,75,75,76+3,76+3,76+3,77+3,81+3,81+3,81+3,77+3,82+3,84+3,84+3,84+3,82+3,82+3,84+3,84+3,84+3,82+3,18+3,Brazil,28,NaN,NaN,NaN,NaN
2391,197061,https://sofifa.com/player/197061/joel-matip/22...,J. Matip,Joël Andre Job Matip,CB,83,83,30500000.0,130000.0,1991-08-08,90,194,9.0,RCB,32.0,Right,2,2,2,Medium/Medium,Lean (185+),56400000.0,"#Aerial Threat, #Tactician",Injury Prone,58.0,46.0,68.0,68.0,85.0,78.0,54,36,84,69,63,22,74,72,50,65,51,84,42,66,75,69,81,48,84,86,48,72,42,80,86,86,81,11,15,10,7,10,NaN,62+2,62+2,62+2,62,64,64,64,62,66+2,66+2,66+2,65+2,72+2,72+2,72+2,65+2,74+2,80+2,80+2,80+2,74+2,76+2,83,83,83,76+2,19+2,Cameroon,30,NaN,NaN,NaN,NaN
2412,239818,https://sofifa.com/player/239818/ruben-santos-...,Rúben Dias,Rúben dos Santos Gato Alves Dias,CB,87,91,102500000.0,170000.0,1997-05-14,82,187,10.0,RCB,3.0,Right,4,2,3,Medium/High,Normal (185+),197300000.0,#Tackling,"Solid Player, Leadership

In [27]:
data["player_tags"].replace("\xa0", "", regex=True, inplace=True)

In [28]:
player_traits_dummies = data.player_traits.str.get_dummies(sep=', ')
player_traits_dummies.columns = ['trait '+col_name for col_name in player_traits_dummies.columns]

player_tags_dummies = data.player_tags.str.get_dummies(sep=', ')
player_tags_dummies.columns = ['tag '+col_name for col_name in player_tags_dummies.columns]

player_positions_dummies = data.player_positions.str.get_dummies(sep=', ')

work_rate_dummies = data.work_rate.str.get_dummies()
work_rate_dummies.columns = ['work rate ' + col_name for col_name in work_rate_dummies.columns]

body_type_dummies = data.body_type.str.get_dummies()
body_type_dummies.columns = ['body type ' + col_name for col_name in body_type_dummies.columns]

preferred_foot_dummies = data.preferred_foot.str.get_dummies()
preferred_foot_dummies.columns = ['preferred foot ' + col_name for col_name in preferred_foot_dummies.columns]

nationality_name_dummies = data.nationality_name.str.get_dummies()
nationality_name_dummies.columns = ['nationality ' + col_name for col_name in nationality_name_dummies.columns]

processed_data = pd.concat(
    [
        data, player_traits_dummies, player_tags_dummies,
        player_positions_dummies, work_rate_dummies, body_type_dummies,
        preferred_foot_dummies, nationality_name_dummies
    ], axis=1
)
# processed_data.drop(columns=["player_traits", "player_tags", "player_positions"], inplace=True)
processed_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,BMI,obese,lean,normal,trait Cautious With Crosses,trait Chip Shot (AI),trait Comes For Crosses,trait Dives Into Tackles (AI),trait Early Crosser,trait Finesse Shot,trait Flair,trait GK Long Throw,trait Giant Throw-in,trait Injury Prone,trait Leadership,trait Long Passer (AI),trait Long Shot Taker (AI),trait Long Throw-in,trait One Club Player,trait Outside Foot Shot,trait Playmaker (AI),trait Power Free-Kick,trait Power Header,trait Rushes Out Of Goal,trait Saves with Feet,trait Solid Player,trait Speed Dribbler (AI),trait Team Player,trait Technical Dribbler (AI),tag #Acrobat,tag #Aerial Threat,tag #Clinical Finisher,tag #Complete Defender,tag #Complete Forward,tag #Complete Midfielder,tag #Crosser,tag #Distance Shooter,tag #Dribbler,tag #Engine,tag #FK Specialist,tag #Playmaker,tag #Poacher,tag #Speedster,tag #Strength,tag #Tackling,tag #Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,work rate High/High,work rate High/Low,work rate High/Medium,work rate Low/High,work rate Low/Low,work rate Low/Medium,work rate Medium/High,work rate Medium/Low,work rate Medium/Medium,body type Lean (170-),body type Lean (170-185),body type Lean (185+),body type Normal (170-),body type Normal (170-185),body type Normal (185+),body type Stocky (170-),body type Stocky (170-185),body type Stocky (185+),body type Unique,preferred foot Left,preferred foot Right,nationality Afghanistan,nationality Albania,nationality Algeria,nationality Andorra,nationality Angola,nationality Antigua and Barbuda,nationality Argentina,nationality Armenia,nationality Australia,nationality Austria,nationality Azerbaijan,nationality Barbados,nationality Belarus,nationality Belgium,nationality Belize,nationality Benin,nationality Bermuda,nationality Bhutan,nationality Bolivia,nationality Bosnia and Herzegovina,nationality Brazil,nationality Bulgaria,nationality Burkina Faso,nationality Burundi,nationality Cameroon,nationality Canada,nationality Cape Verde Islands,nationality Central African Republic,nationality Chad,nationality Chile,nationality China PR,nationality Chinese Taipei,nationality Colombia,nationality Comoros,nationality Congo,nationality Congo DR,nationality Costa Rica,nationality Croatia,nationality Cuba,nationality Curacao,nationality Cyprus,nationality Czech Republic,nationality Côte d'Ivoire,nationality Denmark,nationality Dominican Republic,nationality Ecuador,nationality Egypt,nationality El Salvador,nationality England,nationality Equatorial Guinea,nationality Eritrea,nationality Estonia,nationality Ethiopia,nationality Faroe Islands,nationality Fiji,nationality Finland,nationality France,nationality Gabon,nationality Gambia,nationality Georgia,nationality Germany,nationality Ghana,nationality Gibraltar,nationality Greece,nationality Grenada,nationality Guam,nationality Guatemala,nationality Guinea,nationality Guinea Bissau,nationality Guyana,nationality 

In [29]:
# Convertendo o tipo das colunas
columns_to_evaluate = "ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk".split(",")
for column_to_evaluate in columns_to_evaluate:
    evaluated_column = processed_data[column_to_evaluate].apply(lambda cell: eval(cell))
    processed_data[column_to_evaluate] = evaluated_column

processed_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,BMI,obese,lean,normal,trait Cautious With Crosses,trait Chip Shot (AI),trait Comes For Crosses,trait Dives Into Tackles (AI),trait Early Crosser,trait Finesse Shot,trait Flair,trait GK Long Throw,trait Giant Throw-in,trait Injury Prone,trait Leadership,trait Long Passer (AI),trait Long Shot Taker (AI),trait Long Throw-in,trait One Club Player,trait Outside Foot Shot,trait Playmaker (AI),trait Power Free-Kick,trait Power Header,trait Rushes Out Of Goal,trait Saves with Feet,trait Solid Player,trait Speed Dribbler (AI),trait Team Player,trait Technical Dribbler (AI),tag #Acrobat,tag #Aerial Threat,tag #Clinical Finisher,tag #Complete Defender,tag #Complete Forward,tag #Complete Midfielder,tag #Crosser,tag #Distance Shooter,tag #Dribbler,tag #Engine,tag #FK Specialist,tag #Playmaker,tag #Poacher,tag #Speedster,tag #Strength,tag #Tackling,tag #Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,work rate High/High,work rate High/Low,work rate High/Medium,work rate Low/High,work rate Low/Low,work rate Low/Medium,work rate Medium/High,work rate Medium/Low,work rate Medium/Medium,body type Lean (170-),body type Lean (170-185),body type Lean (185+),body type Normal (170-),body type Normal (170-185),body type Normal (185+),body type Stocky (170-),body type Stocky (170-185),body type Stocky (185+),body type Unique,preferred foot Left,preferred foot Right,nationality Afghanistan,nationality Albania,nationality Algeria,nationality Andorra,nationality Angola,nationality Antigua and Barbuda,nationality Argentina,nationality Armenia,nationality Australia,nationality Austria,nationality Azerbaijan,nationality Barbados,nationality Belarus,nationality Belgium,nationality Belize,nationality Benin,nationality Bermuda,nationality Bhutan,nationality Bolivia,nationality Bosnia and Herzegovina,nationality Brazil,nationality Bulgaria,nationality Burkina Faso,nationality Burundi,nationality Cameroon,nationality Canada,nationality Cape Verde Islands,nationality Central African Republic,nationality Chad,nationality Chile,nationality China PR,nationality Chinese Taipei,nationality Colombia,nationality Comoros,nationality Congo,nationality Congo DR,nationality Costa Rica,nationality Croatia,nationality Cuba,nationality Curacao,nationality Cyprus,nationality Czech Republic,nationality Côte d'Ivoire,nationality Denmark,nationality Dominican Republic,nationality Ecuador,nationality Egypt,nationality El Salvador,nationality England,nationality Equatorial Guinea,nationality Eritrea,nationality Estonia,nationality Ethiopia,nationality Faroe Islands,nationality Fiji,nationality Finland,nationality France,nationality Gabon,nationality Gambia,nationality Georgia,nationality Germany,nationality Ghana,nationality Gibraltar,nationality Greece,nationality Grenada,nationality Guam,nationality Guatemala,nationality Guinea,nationality Guinea Bissau,nationality Guyana,nationality 

In [30]:
data_path = os.sep.join([get_project_root_path(), "preprocessing", "data", f"processed_data_{str(date.today())}.tsv"])
processed_data.to_csv(data_path, index=False, sep="\t")

In [31]:
data_path

'/home/gustavo/fifa22-players-analysis/preprocessing/data/processed_data_2022-06-25.tsv'

In [32]:
processed_data.columns.to_list()

['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'player_positions',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'dob',
 'weight_kg',
 'height_cm',
 'club_team_id',
 'club_position',
 'club_jersey_number',
 'preferred_foot',
 'weak_foot',
 'skill_moves',
 'international_reputation',
 'work_rate',
 'body_type',
 'release_clause_eur',
 'player_tags',
 'player_traits',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_reactions',
 'movement_balance',
 'power_shot_power',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'power_long_shots',
 'mentality_aggression',
 'mentality_interceptions',
 'mentality_positioning',
 'mentality_vision',
 'mentality_penalties',
 'mentality_compos